In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## Читаем файл с параметрами, в качестве индекса задаем дату.

In [3]:
df = pd.read_csv('water_df.csv',index_col='date')
display(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 499400 entries, 2022-10-20 00:10:05 to 2022-10-25 18:53:24
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   cond    499400 non-null  float64
 1   o2      499400 non-null  float64
 2   orp     499400 non-null  float64
 3   tds     499400 non-null  int64  
dtypes: float64(3), int64(1)
memory usage: 19.1+ MB


None

,cond,o2,orp,tds
date,,,,
2022-10-20 00:10:05,359.0,62.9,474.0,369
2022-10-20 00:10:06,353.0,62.9,473.9,369
2022-10-20 00:10:07,345.0,62.7,473.9,370
2022-10-20 00:10:08,342.0,62.8,474.0,370
2022-10-20 00:10:09,339.0,62.8,474.1,368
